# FITTING WITH UNCERTAINTIES (TUTORIAL)
V1.1: Jan 12, 2026 (see changelog at end of document) 

# 1. A linear fit with uncertainties

This section demonstrates two approaches to fitting linear data with uncertainties in both x and y:

1. **2-parameter fit with y uncertainties**: Slope + intercept, using only y-uncertainties.
2. **2-parameter fit with x and y uncertainties**: Slope + intercept, using an iterative method that accounts for x and y uncertainties. This iterative method is often called the effective variance method, the total variance method or just the $\sigma_{tot}$ method.

We will use an example that shows how using this $\sigma_{tot}$ method can improve our fit and result in fitting parameters changing. 

## 1.1 Load python packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit
import pandas as pd # used just for tabular display of data

## 1.2 Our data set

Some pressure sensor calibration data were collected: Sensor Output Voltage (y) vs. Applied Pressure (x)

* **δy:** The output voltage had constant δy = 0.03 V.
* **δx:** Two of the data points were controlled with a precision pressure regulator valve (smaller δx), while the other four were adjusted with a course value such that the readings were fluctuating (larger δx).

In [ ]:
# Data (6 measurements)
x = np.array([0.425, 0.593, 0.832, 1.176, 1.288, 1.488])
y = np.array([3.44, 3.89, 4.04, 4.33, 4.67, 5.08])

# Constant y-uncertainty
y_sigma = np.array([0.03, 0.03, 0.03, 0.03, 0.03, 0.03])

# Varying x-uncertainty
x_sigma = np.array([0.052, 0.012, 0.053, 0.053, 0.014, 0.054])  


# Pandas code being used for nice tabular display
df = pd.DataFrame({"x": x, "δx": x_sigma, "y": y, "δy": y_sigma})
df

## 1.3 Visualize the data

Before fitting, let's visualize the data to make sure a linear fit seems sensible. Notice how some x-measurements are much more precise than others.

In [ ]:
# Plot experimental data
plt.figure(figsize=(8, 6))
plt.errorbar(x, y, xerr=x_sigma, yerr=y_sigma, marker='.', linestyle='', capsize=3)
plt.xlabel("P (bar)")
plt.ylabel("V (V)")
plt.title("Pressure sensor calibration data")
plt.grid(True, alpha=0.3)
plt.show()

**Discussion:** These data are plausibly linear, but seem quite noisy with respect to the size of the y uncertainties.

## 1.4 Define some functions

In [ ]:
# Define fitting function
def line_full(x, m, b):
    """Linear function with slope and intercept"""
    return m * x + b


# Define chi-squared function
def chi_square(fit_function, fit_parameters, x, y, sigma):
    dof = len(x) - len(fit_parameters)
    return np.sum((y - fit_function(x, *fit_parameters))**2 / sigma**2) / dof

## 1.5 Fit 1: 2-Parameter Fit (Slope + Intercept, y-uncertainties only)

Four our first fit in this notebook, we'll fit our pressure sensor data using a linear model (slope + intercept) but only incorporating y-uncertainties in the fit, ignoring x-uncertainties. 

In this section we will
* First, perform this fit, 
* Then we will loop back to look at how to include absolute y-uncertainties in the fit, and
* Finally learn how to extract fitting parameters with uncertainties.

Note that since this is a linear fit, we don't need to provide any initial guesses of our fit parameters, but in a later example of a nonlinear fit we will look at how to include initial guesses of our fit parameters.

**So let's perform the fit!**

In [ ]:
# Fit 1: 2-parameter (slope + intercept), y-uncertainties only
fit_params_1, fit_cov_1 = curve_fit(
    line_full,          # The fitting function, defined above
    x,                  # The array of x-data
    y,                  # The array of y-data
    sigma=y_sigma,      # Optional argument indicating the 
                        #   "one sigma" errors are stored as y_sigma
    absolute_sigma=True # Optional argument indicating the uncertainties indicated by "sigma"
                        #   are absolute errors (e.g., δy), not relative errors (δy/y)
)

# Calculate chi-squared
chi2_1 = chi_square(line_full, fit_params_1, x, y, y_sigma)

# Extract fit parameters and their uncertainties. 
m = fit_params_1[0]
b = fit_params_1[1]
fit_params_sigma_1 = np.sqrt(np.diag(fit_cov_1))
m_sigma = fit_params_sigma_1[0]
b_sigma = fit_params_sigma_1[1]



print("FIT 1: 2-PARAMETER (SLOPE + INTERCEPT, δy ONLY)")
print(f"• Slope:     m = ({m:.4f} ± {m_sigma:.4f}) bar/V")
print(f"• Intercept: b = ({b:.4f} ± {b_sigma:.4f}) V")

# Plot Fit 1 results
fig, ax = plt.subplots(2, 1, figsize=(7, 6), gridspec_kw={'height_ratios': [3, 2]})

# Define smooth x for plotting
x_model = np.linspace(min(x)-0.1, max(x)+0.1, 200)
y_fit1 = line_full(x_model, *fit_params_1)
residual_1 = y - line_full(x, *fit_params_1)

# Top panel: data + fit
ax[0].errorbar(x, y, xerr=x_sigma, yerr=y_sigma, marker='.', linestyle='', 
               label="Data", capsize=3)
ax[0].plot(x_model, y_fit1, '-', linewidth=2, label=f"Fit 1 (χ²={chi2_1:.2f})")
ax[0].set_ylabel("V (V)")
ax[0].set_title("Fit 1: Slope + Intercept (y-uncertainties only)")
ax[0].legend()
ax[0].grid(True, alpha=0.3)

# Bottom panel: residuals
ax[1].errorbar(x, residual_1, yerr=y_sigma, marker='.', linestyle='', capsize=3)
ax[1].hlines(0, np.min(x)-0.1, np.max(x)+0.1, lw=1, alpha=0.8, color="k", linestyle="--")
ax[1].set_xlabel("P (bar)")
ax[1].set_ylabel("Residuals (V)")
ax[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

**Discussion:** We get a fairly high chi-squared, but in looking at the residuals it looks like they are scattered somewhat randomly so it doesn't seem like an alternate model would describe the data better. Perhaps is it that our uncertainties we used in our fit were too low. We will revisit this idea in a later section when we perform our second fit. 

### 1.5.1 Including y-uncertainties in fits using `curve_fit`

Let's look at the format of our call to `curve_fit`, focusing on how the `sigma` and `absolute_sigma` optional arguments are used to include y-uncertainties in our fit.

```python
fit_params_1, fit_cov_1 = curve_fit(
    line_full,          # The fitting function, defined above
    x,                  # The array of x-data
    y,                  # The array of y-data
    sigma=y_sigma,      # Optional argument indicating the 
                        #   "one sigma" errors are stored as y_sigma
    absolute_sigma=True # Optional argument indicating the uncertainties indicated by "sigma"
                        #   are absolute errors (e.g., δy), not relative errors (δy/y)
)
```

As described by the inline comments, in this call we have:

* `sigma=y_sigma`: This argument indicates that our fitting uncertainties, which will be treated as uncertainties in y, are stored in an array `y_sigma`. These are "one sigma" errors, meaning that they represent our regular 68% Confidence Interval style of standard uncertainties.
* `absolute_sigma=True`: This argument indicates that the uncertainties indicated by `sigma` are absolute errors, and not relative errors. Absolute errors are the regular uncertainty representation that you are used to, such as when we state a value with its uncertainty, such as y ± δy. We contrast this with relative uncertainty or relative error, which is a dimensionless representation of uncertainty as a fraction of the value the uncertainty is attached to (e.g., δy/y).

### 1.5.2 Extracting fitting parameter uncertainties

Let's take a moment to understand a bit better how we have been extracting the fitting parameter uncertainties `δm` and `δb` in our previous calls to `curve_fit`. When we call the `curve_fit` function it returns two arrays, which in this case we assign to `fit_params_1` and `fit_cov_1`:

```python
fit_params_1, fit_cov_1 = curve_fit(..)

m = fit_params_1[0]
b = fit_params_1[1]

fit_params_sigma_1 = np.sqrt( np.diag(fit_cov_1) )
m_sigma = fit_params_sigma_1[0]
b_sigma = fit_params_sigma_1[1]
```

Let's look at these two arrays that were returned:
1. `fit_params_1` (called `popt` in the `curve_fit` documentation) returns a 1D array of the optimal values of the parameters resulting from minimization of the sum of the squared residuals. In our example, `fit_params_1[0]` gives us `m` and `fit_params_1[1]` gives us `b`. We know that the order is `m` and then `b` based on the order that these parameters appear in our fitting function: `line_full(x, m, b)`.<br><br>
1. `fitcov2` (called `pcov` in the `curve_fit` documentation) returns a 2D array where the diagnonal of the covariance matrix gives the variance (standard deviation squared) of our fitting parameters and the off-diagnonals communicate the correlations between our fitting parameters. 

In the code above, we can see that we first get the diagonal of the covariance matrix, `np.diag(fit_cov_1)`, and then we  take the square root of this diagonal, `np.sqrt( np.diag(fit_cov_1) )` to get an array of fit parameter uncertainties in the same order as our `fit_params_1` array.

In [ ]:
# Output the covariance matrix
# fit_cov_1

## 1.6 Fit 2: 2-Parameter Fit with the iterative $\sigma_{tot}$ Method

As mentioned at the start of the document, we are using an iterative method that combines the x- and y- uncertainties into an overall effective y-uncertainty ($\sigma_\text{tot}$), which involves converting $\delta x$ into an effective $\delta y_\text{eff}$ using the local slope at that point (aka the derivative of the fit function):

$$\delta y_\text{eff} = \frac{df}{dx}\delta x.$$

Since we are using a linear fitting function in this example, this derivative is simply our best estimate of the slope, `m`. 

Once we have $\delta y_\text{eff}$, our overall y-uncertainty ($\sigma_\text{tot}$) is given by

$$\sigma_\text{tot} = \sqrt{\delta y^2 + \delta y^2_\text{eff} }.$$

These effective uncertainties depend on $\frac{df}{dx}$, such that we need to use an iterative approach to ensure that the fitting parameters are allowed to converge to new values since changing the fitting uncertainty from $\delta y$ to $\sigma_\text{tot}$ will cause small changes in the fitting parameters.

### 1.6.1 Fit 2, Iteration 1 ($\sigma_{tot}$ Method)

In [ ]:
### Fit 2, Iteration 1: Use Fit 1 results as initial dy/dx = m

previous_slope = fit_params_1[0] # From our previous fit
dydx_21 = previous_slope

# Determine sigma_tot
y_sigma_eff = dydx_21 * x_sigma
sigma_tot = np.sqrt(y_sigma**2 + (y_sigma_eff)**2)

# Again, we use pandas to help us display the results in a nice tabular format
df = pd.DataFrame({"x": x, "y": y, "δy": y_sigma, "δx": x_sigma, "δy_eff = mδx": y_sigma_eff, "σ_tot": sigma_tot})
df

**Discussion:** What we notice above is that data points 1 and 4, which had much more precise x values due to the precision pressure regulator valve, end up having $\sigma_{tot}$ values that are not significantly higher than their original $\delta_y$ values. Conversely, the other data points have significantly larger $\sigma_{tot}$ due the their relatively larger values of $\delta_x$. 

Before proceeding, let's plot these data with the original $\delta_y$ and updated $\sigma_{tot}$ values, where we can see this difference in the $\sigma_{tot}$ uncertainties for the two types of data points.

In [ ]:
plt.figure(figsize=(8, 6))
plt.errorbar(x, y, yerr=y_sigma, marker='.', linestyle='', capsize=3, color='red', label='Data with δy')
plt.errorbar(x, y, yerr=sigma_tot, linestyle='', marker='.', capsize=3, color='black', label='Data with σ_tot')
plt.xlabel("P (bar)")
plt.ylabel("V (V)")
plt.title("Comparison of δy and σ_tot based on iteration 1 estimate of dy/dx")
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

Now we perform the first iteration of this fit.

In [ ]:
# We update our uncertainty to sigma=sigma_tot when we call curve_fit
fit_params_21, fit_cov_21 = curve_fit(
    line_full, x, y, sigma=sigma_tot, absolute_sigma=True)

# Useful paraemters from the fit
fit_params_sigma_21 = np.sqrt(np.diag(fit_cov_21))

# Plot Fit 2, Iteration results and compare to Fit 1

fig, ax = plt.subplots(2, 1, figsize=(7, 6), gridspec_kw={'height_ratios': [3, 2]})

# Define smooth x for plotting
y_fit_21 = line_full(x_model, *fit_params_21)
residual_21 = y - line_full(x, *fit_params_21)

# Top panel: data + fit
ax[0].errorbar(x, y, yerr=sigma_tot, marker='.', linestyle='', 
               label="Data with σ_tot", capsize=3)
ax[0].plot(x_model, y_fit1, '--', linewidth=2, label=f"Fit 1")
ax[0].plot(x_model, y_fit_21, 'k-', linewidth=2, label=f"Fit 2, Iteration 1")
ax[0].set_ylabel("V (V)")
ax[0].set_title("Fit 2, Iteration 1: Slope + Intercept (σ_tot uncertainties)")
ax[0].legend()
ax[0].grid(True, alpha=0.3)

# Bottom panel: residuals
ax[1].errorbar(x, residual_21, yerr=sigma_tot, marker='.', linestyle='', capsize=3)
ax[1].hlines(0, np.min(x)-0.1, np.max(x)+0.1, lw=1, alpha=0.8, color="k", linestyle="--")
ax[1].set_xlabel("P (bar)")
ax[1].set_ylabel("Residuals (V)")
ax[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("FIT 2: ITERATION 1 (σ_tot METHOD)")
print(f"• Slope:     m = ({fit_params_21[0]:.4f} ± {fit_params_sigma_21[0]:.4f}) bar/V") 
print(f"• Intercept: b = ({fit_params_21[1]:.4f} ± {fit_params_sigma_21[1]:.4f}) V")
print(f"\nChange from Fit 1 to Fit 2, Iteration 1:")
print(f"• Δm = {abs(fit_params_21[0] - fit_params_1[0]):.4f}"
      f" ({abs(fit_params_21[0] - fit_params_1[0])/fit_params_1[0]*100:.2f}%)")
print(f"• Δb = {abs(fit_params_21[1] - fit_params_1[1]):.4f}"
      f" ({abs(fit_params_21[1] - fit_params_1[1])/fit_params_1[1]*100:.2f}%)")

**Discussion:** The paramaters have changed significantly once we introduced σ_tot. We should expect needing another one or two iterations before these parameters start to converge.

### 1.6.2 Fit 2, Iteration 2 ($\sigma_{tot}$ Method)

We do the same process as we did for Iteration 1 except we use our new value of slope (dydx) to estimate dy_eff and σ_tot. We expect our updated fit parameters after Iteration 2 to be somewhat close to what they were after Iteration 1.

In [ ]:
### Fit 2, Iteration 2: Use Fit 2.1 results as initial dy/dx = m

# Update dydx to be used for sigma_tot
dydx_22 = fit_params_21[0] # From Fit 2, Iteration 1
 
# Determine updated sigma_tot since our best-estimate of the slope has changed
y_sigma_eff_22 = dydx_22 * x_sigma
sigma_tot = np.sqrt(y_sigma**2 + (y_sigma_eff_22)**2)

# Call curve fit
fit_params_22, fit_cov_22 = curve_fit(line_full, x, y, sigma=sigma_tot, absolute_sigma=True)

# Useful paraemters from the fit
fit_params_sigma_22 = np.sqrt(np.diag(fit_cov_22))

# Plot Iteration 2 to compare to Iteration 1

fig, ax = plt.subplots(2, 1, figsize=(7, 6), gridspec_kw={'height_ratios': [3, 2]})

# Define smooth x for plotting
y_fit_22 = line_full(x_model, *fit_params_22)
residual_22 = y - line_full(x, *fit_params_22)

# Top panel: data + fit
ax[0].errorbar(x, y, yerr=sigma_tot, marker='.', linestyle='', 
               label="Data with σ_tot", capsize=3)
ax[0].plot(x_model, y_fit_21, 'k--', linewidth=2, label=f"Iteration 1")
ax[0].plot(x_model, y_fit_22, 'r-', linewidth=2, label=f"Iteration 2")
ax[0].set_ylabel("V (V)")
ax[0].set_title("Fit 2, Iteration 2: Slope + Intercept (σ_tot uncertainties)")
ax[0].legend()
ax[0].grid(True, alpha=0.3)

# Bottom panel: residuals
ax[1].errorbar(x, residual_22, yerr=sigma_tot, marker='.', linestyle='', capsize=3)
ax[1].hlines(0, np.min(x)-0.1, np.max(x)+0.1, lw=1, alpha=0.8, color="k", linestyle="--")
ax[1].set_xlabel("P (bar)")
ax[1].set_ylabel("Residuals (V)")
ax[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("FIT 2: ITERATION 2 (σ_tot METHOD)")
print(f"• Slope     m = ({fit_params_22[0]:.4f} ± {fit_params_sigma_22[0]:.4f}) bar/V")
print(f"• Intercept b = ({fit_params_22[1]:.4f} ± {fit_params_sigma_22[1]:.4f}) V")
print(f"\nChange from Iteration 1 to Iteration 2:")
print(f"• Δm = {abs(fit_params_22[0] - fit_params_21[0]):.4f}"
      f" ({abs(fit_params_22[0] - fit_params_21[0])/fit_params_21[0]*100:.2f}%)")
print(f"• Δb = {abs(fit_params_22[1] - fit_params_21[1]):.4f}"
      f" ({abs(fit_params_22[1] - fit_params_21[1])/fit_params_21[1]*100:.2f}%)")

**Discussion:** Given that our fit parameters are now changing by less than 1% each, we can expect that a third iteration would result in negligible changes to these fit parameters. However, for the purposes of this example, let's confirm that this is true.

### 1.6.3 Fit 2, Iteration 3 ($\sigma_{tot}$ Method)

In [ ]:
### Fit 2, Iteration 3: Use Fit 2.2 results as initial dy/dx = m

# Update dydx to be used for sigma_tot
dydx_23 = fit_params_22[0] # From Iteration 2
 
# Determine updated sigma_tot since our best-estimate of the slope has changed
y_sigma_eff_23 = dydx_23 * x_sigma
sigma_tot = np.sqrt(y_sigma**2 + (y_sigma_eff_23)**2)

# Call curve fit
fit_params_23, fit_cov_23 = curve_fit(line_full, x, y, sigma=sigma_tot, absolute_sigma=True)

# Useful paraemters from the fit
fit_params_sigma_23 = np.sqrt(np.diag(fit_cov_23))

# Plot Iteration 3 to compare to Iteration 2

fig, ax = plt.subplots(2, 1, figsize=(7, 6), gridspec_kw={'height_ratios': [3, 2]})

# Define smooth x for plotting
y_fit_23 = line_full(x_model, *fit_params_23)
residual_23 = y - line_full(x, *fit_params_23)

# Top panel: data + fit
ax[0].errorbar(x, y, yerr=sigma_tot, marker='.', linestyle='', 
               label="Data with σ_tot", capsize=3)
ax[0].plot(x_model, y_fit_22, 'r--', linewidth=2, label=f"Iteration 2")
ax[0].plot(x_model, y_fit_23, 'g-', linewidth=2, label=f"Iteration 3")
ax[0].set_ylabel("V (V)")
ax[0].set_title("Fit 2, Iteration 3: Slope + Intercept (σ_tot uncertainties)")
ax[0].legend()
ax[0].grid(True, alpha=0.3)

# Bottom panel: residuals
ax[1].errorbar(x, residual_23, yerr=sigma_tot, marker='.', linestyle='', capsize=3)
ax[1].hlines(0, np.min(x)-0.1, np.max(x)+0.1, lw=1, alpha=0.8, color="k", linestyle="--")
ax[1].set_xlabel("P (bar)")
ax[1].set_ylabel("Residuals (V)")
ax[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("FIT 2: ITERATION 3 (σ_tot METHOD)")
print(f"• Slope     m = ({fit_params_23[0]:.4f} ± {fit_params_sigma_23[0]:.4f}) bar/V")
print(f"• Intercept b = ({fit_params_23[1]:.4f} ± {fit_params_sigma_23[1]:.4f}) V")
print(f"\nChange from Iteration 2 to Iteration 3:")
print(f"• Δm = {abs(fit_params_23[0] - fit_params_22[0]):.4f}"
      f" ({abs(fit_params_23[0] - fit_params_22[0])/fit_params_22[0]*100:.2f}%)")
print(f"• Δb = {abs(fit_params_23[1] - fit_params_22[1]):.4f}"
      f" ({abs(fit_params_23[1] - fit_params_22[1])/fit_params_22[1]*100:.2f}%)")

**Discussion:** As we expected, the changes in our fit parameters in this third iteration were negligible (< 0.1%). The residuals still show good scatter, but many points are further away from the R = 0 line so we expect our final chi-squared to remain somewhat high (and we see below that it is). 

In [ ]:
chi2_2 = chi_square(line_full, fit_params_23, x, y, sigma_tot)
print(f"Fit 2 (after 3 iterations): χ² = {chi2_2:.2f}  (using δy = σ_tot)")

**Summary:** This section demonstrated how we can account for x and y uncertainties by using the the instanteous slope at each data point to estimate an effective y uncertainty based on the x uncertainty, and to then combine those two uncertainties into an overall uncertainty σ_tot. Because the instantaneous slopes at each data point update after each fit attempt, this process often takes multiple iteractions for the parameters to converge to a stable value

# 2. A nonlinear fit with uncertainties

In Section 1 we looked at fitting a linear model to data with uncertainties in both x and y, demonstrating the iterative σ_tot method. Now in Section 2, we'll extend these concepts to **nonlinear fitting**, where the fitting function is not a simple straight line.

For this example, we'll fit an **exponential decay with background** model to radiation shielding data:

$$R(x) = R_0 e^{-\mu x} + R_B$$

where:
- $R_0$ is the initial count rate (counts/min)
- $\mu$ is the attenuation coefficient (mm<sup>-1</sup>)
- $R_B$ is the background count rate (counts/min)
- $x$ is the shielding thickness (mm)

**Key difference from linear fitting**: Since this model is nonlinear in its parameters, we must provide **initial guesses** for the parameters when calling `curve_fit`. The algorithm will then iteratively refine these guesses to minimize the sum of squared residuals.

We'll demonstrate two fitting approaches:
1. **Fit 1**: 3-parameter fit using y-uncertainties only (ignoring x-uncertainties)
2. **Fit 2**: 3-parameter fit using the iterative σ_tot method (incorporating both x- and y-uncertainties)

## 2.1 Our nonlinear radiation countrate data set

This is a fabricated data set for a radiation shielding experiment where a radioactive source emits radiation through varying thicknesses of shielding material. The detector measures the count rate (counts per minute) as a function of shielding thickness.

**Physical expectations**:
- Count rate should decrease exponentially with increasing shielding thickness
- There should be a background count rate R<sub>B</sub> that persists even with thick shielding (cosmic rays, environmental radiation, detector noise)

**Uncertainties**:
- **δR** (y-uncertainties): From Poisson counting statistics and a small contribution from timing uncertainties
- **δx** (x-uncertainties): From measurement precision in shielding thickness.

In [ ]:
# Shielding thickness (mm)
x = np.array([0., 1.02, 1.95, 3.08, 3.91, 5.12, 5.87, 7.15, 8.22, 8.78])

# Count rate (counts/min)
R = np.array([526, 382, 300, 240, 162, 123, 113, 82, 56, 52])

# x-uncertainties (thickness measurement precision)
x_sigma = np.array([0.0, 0.10, 0.14, 0.12, 0.17, 0.14, 0.19, 0.16, 0.21, 0.18])

# y-uncertainties
R_sigma = np.array([11.5, 9.8, 8.7, 7.8, 6.4, 5.6, 5.3, 4.6, 3.8, 3.6])

# Display data in table format using pandas
df = pd.DataFrame({
    "x (mm)": x,
    "δx": x_sigma,
    "R (counts/min)": R,
    "δR": R_sigma
})
df

## 2.2 Visualize the data

Before fitting, let's examine our data to confirm that an exponential decay model seems appropriate. We expect to see the count rate decreasing rapidly at first, then leveling off toward the background level.

In [ ]:
plt.figure(figsize=(8, 6))
plt.errorbar(x, R, xerr=x_sigma, yerr=R_sigma, marker='.', linestyle='', capsize=3)
plt.xlabel("Shielding thickness (mm)")
plt.ylabel("Count rate (counts/min)")
plt.title("Radiation shielding experimental data")
plt.grid(True, alpha=0.3)
plt.show()

## 2.3 Define the fitting functions

Our exponential decay model with background is:

$$R(x) = R_0 e^{-\mu x} + R_B$$

**Parameters**:
- **R<sub>0</sub>**: Initial count rate when x = 0 (counts/min)
- **μ**: Attenuation coefficient of the shielding related to this type of radiation(mm<sup>-1</sup>)
- **R<sub>B</sub>**: Background count rate that persists at large x (counts/min)

Since this is a nonlinear fitting function (nonlinear in the parameters R<sub>0</sub>, μ, and R<sub>B</sub>), we'll need to provide initial guesses for these parameters when we call `curve_fit`.

We also define the derivative df/dx, which we'll need for calculating σ_tot later:

$$\frac{dR}{dx} = -R_0 \mu \, e^{-\mu x}$$

In [ ]:
def decay_with_background(x, R0, mu, B):
    """3-parameter: exponential decay with background
    R(x) = R₀ × exp(-μx) + B

    Parameters:
    -----------
    R0 : Initial count rate (counts/min)
    mu : Attenuation coefficient (1/mm)
    B : Background count rate (counts/min)
    """
    return R0 * np.exp(-mu * x) + B

def decay_derivative(x, R0, mu, B):
    """Derivative df/dx for σ_tot calculation
    Used to convert x-uncertainties to effective y-uncertainties
    
    dR/dx = -R₀μ exp(-μx)
    """
    return -R0 * mu * np.exp(-mu * x)

## 2.4 Fit 1: 3-Parameter Fit (R₀, μ, B with regular y-uncertainties only)

For our first fit of the nonlinear model, we'll use only the y-uncertainties (δR from Poisson statistics) and ignore the x-uncertainties in shielding thickness.

**Key requirement**: Since this is a nonlinear fitting function, we must provide initial guesses for the three parameters (R₀, μ, B) using the `p0` argument in `curve_fit`.

### 2.4.1 Estimating initial guesses

How can we estimate reasonable initial guesses for the parameters?

- **R₀ (initial count rate)**: Should be close to the maximum count rate in our data minus our background count rate, since R<sub>0</sub> represents the count rate at x = 0
- **B (background)**: Should be close to the minimum count rate, representing the asymptotic value as x → ∞
- **μ (attenuation coefficient)**: With exponential decays, this coefficient represent how quickly the quantity decays to 1/e = 37% of its value. Looking at the graph, it looks like the count rate drops to apprximately 1/3 of its original value after approximately 3.5 mm, so a reasonable initial guess for μ = 1/(3.5mm) = 0.29/mm.

Let's estimate these from our data and visualize how well our initial guess matches:

In [ ]:
# Estimate initial guesses from data
B_guess = np.min(R) # Background level
R0_guess = np.max(R)-np.min(R) # Initial count rate minus background
mu_guess = 1/3.5 # Attenuation coefficient (1/mm)

print("Initial parameter guesses:")
print(f"• R_0 ≈ {R0_guess:.1f} counts/min")
print(f"• μ  ≈ {mu_guess:.2f} mm⁻¹")
print(f"• R_B  ≈ {B_guess:.1f} counts/min")

# Visualize initial guess
plt.figure(figsize=(8, 6))
plt.errorbar(x, R, yerr=R_sigma, marker='.', linestyle='', capsize=3, label="Data")
x_model = np.linspace(min(x), max(x), 200)
R_guess = decay_with_background(x_model, R0_guess, mu_guess, B_guess)
plt.plot(x_model, R_guess, 'g--', linewidth=2, label="Initial guess")
plt.xlabel("Shielding thickness (mm)")
plt.ylabel("Count rate (counts/min)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### 2.4.2 Perform the fit

Now we perform the fit using `curve_fit` with our initial guesses. We use `sigma=R_sigma` to incorporate the uncertainties and `absolute_sigma=True` to ensure proper uncertainty scaling.

In [ ]:
# Fit 1: 3-parameter, y-uncertainties only
fit_params_1, fit_cov_1 = curve_fit(
    decay_with_background, x, R,
    p0=[R0_guess, mu_guess, B_guess],
    sigma=R_sigma,
    absolute_sigma=True
)

# Extract parameters and uncertainties
fit_params_sigma_1 = np.sqrt(np.diag(fit_cov_1))
chi2_1 = chi_square(decay_with_background, fit_params_1, x, R, R_sigma)

# Display results
print("FIT 1: 3-PARAMETER (R₀, μ, B with δR only)")
print(f"• Reduced χ² = {chi2_1:.3f}")
print(f"• R₀ = ({fit_params_1[0]:.2f} ± {fit_params_sigma_1[0]:.2f}) counts/min")
print(f"• μ  = ({fit_params_1[1]:.4f} ± {fit_params_sigma_1[1]:.4f}) mm⁻¹")
print(f"• B  = ({fit_params_1[2]:.2f} ± {fit_params_sigma_1[2]:.2f}) counts/min")

# Plot results
fig, ax = plt.subplots(2, 1, figsize=(7, 6), gridspec_kw={'height_ratios': [3, 2]})

x_model = np.linspace(min(x)-0.1, max(x)+0.1, 200)
R_fit1 = decay_with_background(x_model, *fit_params_1)
residual_1 = R - decay_with_background(x, *fit_params_1)

# Top panel: data + fit
ax[0].errorbar(x, R, xerr=x_sigma, yerr=R_sigma, marker='.', linestyle='',
               label="Data", capsize=3)
ax[0].plot(x_model, R_fit1, '-', linewidth=2, label=f"Fit 1 (χ²={chi2_1:.2f})")
ax[0].set_ylabel("Count rate (counts/min)")
ax[0].set_title("Fit 1: Exponential decay with background (y-uncertainties only)")
ax[0].legend()
ax[0].grid(True, alpha=0.3)

# Bottom panel: residuals
ax[1].errorbar(x, residual_1, yerr=R_sigma, marker='.', linestyle='', capsize=3)
ax[1].hlines(0, np.min(x)-0.1, np.max(x)+0.1, lw=1, alpha=0.8, color="k", linestyle="--")
ax[1].set_xlabel("Shielding thickness (mm)")
ax[1].set_ylabel("Residuals")
ax[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

**Discussion**: The fit looks quite good, and the residuals appear randomly scattered. However, we haven't yet incorporated the x-uncertainties (thickness measurement errors). In the next section, we'll use the iterative σ_tot method to properly account for both x- and y-uncertainties.

## 2.5 Fit 2: 3-Parameter Fit with Iterative σ_tot Method

Now we incorporate the x-uncertainties using the iterative σ_tot method, just as we did in Section 1. The total uncertainty is:

$$\sigma_{tot} = \sqrt{\delta R^2 + \left(\frac{dR}{dx} \delta x\right)^2}$$

For our exponential decay model, the derivative is:

$$\frac{dR}{dx} = -R_0 \mu \, e^{-\mu x}$$

Since σ_tot depends on the fitted parameters (R₀ and μ), we must iterate:
1. Use Fit 1 results to calculate an initial σ_tot
2. Fit with this σ_tot to get updated parameters
3. Recalculate σ_tot with the new parameters
4. Repeat until parameters converge

### 2.5.1 Iteration 1

We start by using the Fit 1 results to calculate the derivative and σ_tot.

In [ ]:
# Use Fit 1 results as initial guess
previous_params = fit_params_1
dRdx = decay_derivative(x, *previous_params)

# Calculate σ_tot
R_sigma_eff = np.abs(dRdx) * x_sigma
sigma_tot = np.sqrt(R_sigma**2 + R_sigma_eff**2)

# Perform fit
fit_params_21, fit_cov_21 = curve_fit(
    decay_with_background, x, R,
    p0=previous_params,
    sigma=sigma_tot,
    absolute_sigma=True
)

fit_params_sigma_21 = np.sqrt(np.diag(fit_cov_21))

print("FIT 2: ITERATION 1 (σ_tot METHOD)")
print(f"• R₀ = ({fit_params_21[0]:.2f} ± {fit_params_sigma_21[0]:.2f}) counts/min")
print(f"• μ  = ({fit_params_21[1]:.4f} ± {fit_params_sigma_21[1]:.4f}) mm⁻¹")
print(f"• B  = ({fit_params_21[2]:.2f} ± {fit_params_sigma_21[2]:.2f}) counts/min")
print(f"\nChange from Fit 1:")
print(f"• ΔR₀ = {abs(fit_params_21[0] - fit_params_1[0]):.2f}"
      f" ({abs(fit_params_21[0] - fit_params_1[0])/fit_params_1[0]*100:.2f}%)")
print(f"• Δμ  = {abs(fit_params_21[1] - fit_params_1[1]):.4f}"
      f" ({abs(fit_params_21[1] - fit_params_1[1])/fit_params_1[1]*100:.2f}%)")
print(f"• ΔB  = {abs(fit_params_21[2] - fit_params_1[2]):.2f}"
      f" ({abs(fit_params_21[2] - fit_params_1[2])/fit_params_1[2]*100:.2f}%)")

# Plot
fig, ax = plt.subplots(2, 1, figsize=(7, 6), gridspec_kw={'height_ratios': [3, 2]})

R_fit21 = decay_with_background(x_model, *fit_params_21)
residual_21 = R - decay_with_background(x, *fit_params_21)


ax[0].errorbar(x, R, yerr=sigma_tot, marker='.', linestyle='',
               label="Data with σ_tot", capsize=3)
ax[0].plot(x_model, R_fit1, '--', linewidth=2, label=f"Fit 1")
ax[0].plot(x_model, R_fit21, 'r-', linewidth=2, label=f"Fit 2, Iter 1")
ax[0].set_ylabel("Count rate (counts/min)")
ax[0].set_title("Fit 2: Exponential decay with σ_tot method")
ax[0].legend()
ax[0].grid(True, alpha=0.3)

ax[1].errorbar(x, residual_21, yerr=sigma_tot, marker='.', linestyle='', capsize=3)
ax[1].hlines(0, np.min(x)-0.1, np.max(x)+0.1, lw=1, alpha=0.8, color="k", linestyle="--")
ax[1].set_xlabel("Shielding thickness (mm)")
ax[1].set_ylabel("Residuals")
ax[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

**Discussion:** This already shows a minimal change with respect to Fit 1 other than the background count rate. Let's do a second iteration.

### 2.5.2 Iteration 2

Now we use the Iteration 1 results to recalculate σ_tot and fit again.

In [ ]:
# Update using Iteration 1 results
dRdx = decay_derivative(x, *fit_params_21)
R_sigma_eff = np.abs(dRdx) * x_sigma
sigma_tot = np.sqrt(R_sigma**2 + R_sigma_eff**2)

# Perform fit
fit_params_22, fit_cov_22 = curve_fit(
    decay_with_background, x, R,
    p0=fit_params_21,
    sigma=sigma_tot,
    absolute_sigma=True
)

fit_params_sigma_22 = np.sqrt(np.diag(fit_cov_22))
chi2_22 = chi_square(decay_with_background, fit_params_22, x, R, sigma_tot)

print("FIT 2: ITERATION 2 (σ_tot METHOD)")
print(f"• Reduced χ² = {chi2_22:.3f}")
print(f"• R₀ = ({fit_params_22[0]:.2f} ± {fit_params_sigma_22[0]:.2f}) counts/min")
print(f"• μ  = ({fit_params_22[1]:.4f} ± {fit_params_sigma_22[1]:.4f}) mm⁻¹")
print(f"• B  = ({fit_params_22[2]:.2f} ± {fit_params_sigma_22[2]:.2f}) counts/min")
print(f"\nChange from Iteration 1:")
print(f"• ΔR₀ = {abs(fit_params_22[0] - fit_params_21[0]):.2f}"
      f" ({abs(fit_params_22[0] - fit_params_21[0])/fit_params_21[0]*100:.2f}%)")
print(f"• Δμ  = {abs(fit_params_22[1] - fit_params_21[1]):.4f}"
      f" ({abs(fit_params_22[1] - fit_params_21[1])/fit_params_21[1]*100:.2f}%)")
print(f"• ΔB  = {abs(fit_params_22[2] - fit_params_21[2]):.2f}"
      f" ({abs(fit_params_22[2] - fit_params_21[2])/fit_params_21[2]*100:.2f}%)")

# Plot
fig, ax = plt.subplots(2, 1, figsize=(7, 6), gridspec_kw={'height_ratios': [3, 2]})

R_fit22 = decay_with_background(x_model, *fit_params_22)
residual_22 = R - decay_with_background(x, *fit_params_22)

ax[0].errorbar(x, R, yerr=sigma_tot, marker='.', linestyle='',
               label="Data with σ_tot", capsize=3)
ax[0].plot(x_model, R_fit21, 'k--', linewidth=2, label=f"Fit 2, Iter 1")
ax[0].plot(x_model, R_fit22, 'r-', linewidth=2, label=f"Fit 2, Iter 2")
ax[0].set_ylabel("Count rate (counts/min)")
ax[0].set_title("Fit 2: Exponential decay with σ_tot method")
ax[0].legend()
ax[0].grid(True, alpha=0.3)

ax[1].errorbar(x, residual_22, yerr=sigma_tot, marker='.', linestyle='', capsize=3)
ax[1].hlines(0, np.min(x)-0.1, np.max(x)+0.1, lw=1, alpha=0.8, color="k", linestyle="--")
ax[1].set_xlabel("Shielding thickness (mm)")
ax[1].set_ylabel("Residuals")
ax[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

**Discussion:** The fit parameters have converged after two iterations, with the change in the background count rate being less than 0.1% on this iteration. 

# Changelog

V1.1: Fixed y-intercept reporting in fit 1